# Task 2: Deep Learning Algorithm for Denoising

Locate a dataset containing both noisy and noise-free multiexponential decay curves using the provided link:

https://drive.google.com/file/d/11glr0L9CnnOfX5sG8zrzXyjJLWgQsUIL/view?usp=sharing.

Suggest simple and straightforward deep learning algorithm to perform supervised denoising of these curves. Choose a validation strategy for your selected method. Select and justify a metric to evaluate the model's performance. Save the denoised curves from the validation dataset as .csv files. Create a train/test demonstration in a Jupyter notebook with basic result visualization.

The dataset includes files named, for example, “cor_36_5_0.csv”, “cor_36_5_1.csv”, “cor_36_5_2.csv”, etc., where “cor_36_5” serves as a file ID, and the number preceding '.csv' (ranging from 0 to 9) represents an arbitrary noise level. Consequently, the file names ending in “0.csv” are considered noise-free targets.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os


direct = "/content/drive/MyDrive/Colab Notebooks/ML_uni/data_d"


## Зчитування файлів та збереження значень в одному DataFrame

In [2]:
# Сортуємо файли та зберігаємо їхній вміст в один DataFrame

dictionary = {f"{n} Noise": [] for n in range(10)}


files = os.listdir(direct)
files = sorted(files)

for i in range(10):
  for file_name in files:
    if file_name.endswith(f"{i}.csv"):
      dictionary[f"{i} Noise"].append(pd.read_csv(os.path.join(direct, file_name)).values)

print(dictionary)

data_files = pd.DataFrame(dictionary)

data_files

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,0 Noise,1 Noise,2 Noise,3 Noise,4 Noise,5 Noise,6 Noise,7 Noise,8 Noise,9 Noise
0,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[2.0079058373783016e-06], [-5.913955291041794...","[[-1.2817367995624436e-06], [2.344011722773164...","[[2.0036033713386737e-06], [6.824707812502529e...","[[4.3581424041258216e-06], [6.569006897211351e...","[[-2.4964635402119328e-06], [3.032434168840905...","[[-7.437716727601282e-06], [-2.730730891096025...","[[5.920086963248883e-06], [3.077108989466493e-...","[[-3.3282096884895327e-06], [9.862803290611444...","[[4.794858948905414e-06], [-1.7004199106781814..."
1,"[[0.7644540852813445], [0.7642527177687716], [...","[[0.7688863274157571], [0.7468303162052349], [...","[[0.7649276588893466], [0.770332898190035], [0...","[[0.7727587348690182], [0.7645684025088355], [...","[[0.7594440036707844], [0.7667598683124744], [...","[[0.7419985415614591], [0.7294945175890569], [...","[[0.6975553056221436], [0.7767322040581865], [...","[[0.8567395093988662], [0.673512990015952], [0...","[[0.8204599679320513], [0.8071188862347569], [...","[[0.8001473519068855], [0.6937056812891452], [..."
2,"[[0.4268325933027093], [0.4264230210485995], [...","[[0.41025000556813], [0.42787505098278006], [0...","[[0.43965289529787216], [0.4247800705257544], ...","[[0.4883475349965597], [0.41473060389163285], ...","[[0.410936277974471], [0.4625441668885077], [0...","[[0.4436450307434035], [0.42878954201087105], ...","[[0.48556115644585607], [0.4255441333571736], ...","[[0.4238191532179412], [0.337980199332346], [0...","[[0.3889988071996867], [0.4557921568268791], [...","[[0.45334425013647583], [0.35186265590351384],..."
3,"[[0.6619496011372515], [0.6597226329889204], [...","[[0.6602655533253672], [0.6599275492461744], [...","[[0.6188817908803392], [0.6445997305502587], [...","[[0.6515885215379433], [0.6740237625108716], [...","[[0.6483021814792684], [0.6002161557037277], [...","[[0.6643542337804553], [0.6071844906738044], [...","[[0.6920288984688425], [0.6728737632158922], [...","[[0.7111239834238725], [0.69397297965473], [0....","[[0.6793156263624065], [0.6763683586784089], [...","[[0.6519041021013476], [0.7815955309978938], [..."
4,"[[0.5578612863643138], [0.5416321382453284], [...","[[0.5589287379924416], [0.5444473568592393], [...","[[0.5710365825425862], [0.5314548128793812], [...","[[0.5407055093964194], [0.551856385282825], [0...","[[0.5522283734551637], [0.6073331092657945], [...","[[0.5504777342705803], [0.5818240374956787], [...","[[0.6319990944946621], [0.5712863548859902], [...","[[0.5131024131607763], [0.5531626632035707], [...","[[0.6940884744989008], [0.5074893690028076], [...","[[0.4018805890095674], [0.420039054199893], [0..."
...,...,...,...,...,...,...,...,...,...,...
495,"[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0....","[[-1.9680689504286642e-07], [1.447336756913718...","[[-4.6673691064085867e-08], [1.898146442942450...","[[-1.1787339391197616e-06], [3.690842679168078...","[[4.1938804614004526e-06], [2.6661565204193684...","[[-8.402551551871827e-06], [-2.067246195871271...","[[-6.528660077798372e-06], [7.835116045519457e...","[[-1.6335194825365288e-05], [6.296640600755455...","[[6.4907496375137135e-06], [-6.689199082975791...","[[7.271870538039312e-08], [5.450022888861933e-..."
496,"[[0.20954738013643784], [0.19047838931634392],...","[[0.21544321047588888], [0.19545694371288488],...","[[0.20230564686122293], [0.18983971408225517],...","[[0.19593161616471427], [0.1949750396838869], ...","[[0.2137134885489028], [0.2234298639608308], [...","[[0.21651117171311932], [0.23080651981074884],...","[[0.2233935254997662], [0.21076031591793085], ...","[[0.19863895414558908], [0.21918439355548447],...","[[0.28159763141440364], [0.19665724189922135],...","[[0.18728988760347104], [0.149449905530973], [..."
497,"[[0.2650240565850747], [0.24703059015176937], ...","[[0.2648120247940753], [0.24396767315362672], ...","[[0.28564742981179064], [0.24332472183009385],...","[[0.26838473585335587], [0.21722247680005355],...","[[0.

In [124]:
print(f"Shape of dataset: {data_files.shape}")
first_column = data_files["0 Noise"]
print(f"Shape of one cell: {first_column.iloc[1].shape}")

Shape of dataset: (500, 10)
Shape of one cell: (119, 1)


In [125]:
X_data = data_files.drop(["9 Noise"], axis=1)
y_data = data_files["9 Noise"]

print(X_data.shape, y_data.shape)

(500, 9) (500,)


In [188]:
data_dict = {"Noise": []}
X_dict = {f"{n} Non-Noise": [] for n in range(9)}

# for i in X_data["0 Noise"]:
#   print(i)
print(X_data["0 Noise"][0].shape)

for n in range(9):
  for i in X_data[f"{n} Noise"][:]:
    X_dict[f"{n} Non-Noise"].extend(i)

for i in y_data:
  data_dict["Noise"].extend(i)


new_data_non = pd.DataFrame(X_dict)
new_data_non

(119, 1)


,0 Non-Noise,1 Non-Noise,2 Non-Noise,3 Non-Noise,4 Non-Noise,5 Non-Noise,6 Non-Noise,7 Non-Noise,8 Non-Noise
0,[0.0],[2.0079058373783016e-06],[-1.2817367995624436e-06],[2.0036033713386737e-06],[4.3581424041258216e-06],[-2.4964635402119328e-06],[-7.437716727601282e-06],[5.920086963248883e-06],[-3.3282096884895327e-06]
1,[0.0],[-5.913955291041794e-07],[2.3440117227731644e-06],[6.824707812502529e-06],[6.569006897211351e-06],[3.032434168840905e-06],[-2.730730891096025e-06],[3.077108989466493e-06],[9.862803290611444e-06]
2,[0.0],[3.3141555733890324e-07],[-3.893218547969645e-07],[4.256933748513527e-06],[3.74755432138715e-07],[4.573981192370317e-06],[8.405934340124155e-06],[-4.489462100876221e-06],[-1.5311527184787625e-05]
3,[0.0],[8.637197721584479e-07],[4.1690609278661995e-06],[-2.2030139779657684e-06],[1.3280937083139418e-06],[-1.1713320668248904e-06],[2.146654905874661e-06],[1.05082938712003e-05],[-7.626913618217552e-06]
4,[0.0],[-9.223602690169983e-08],[7.624650019529575e-08],[6.365160110628065e-07],[2.4244862247536965e-06],[1.9512180186323465e-06],[9.104707178936195e-06],[-2.3321466608201718e-06],[-5.113274569414662e-06]
...,...,...,...,...,...,...,...,...,...
59495,[6.661903366792233e-07],[-0.002668239675760908],[0.008169984150876166],[0.016740765766149423],[-0.002294161816242161],[-0.00026975374682117355],[0.011871699887251272],[-0.020556863121391003],[0.0020761000789510983]
59496,[4.879256639892813e-07],[-0.004265290998220624],[-0.016472211054696494],[-0.011909984576074043],[-0.000701709462919735],[0.006876162020719971],[-0.02845587063977215],[-0.007545923541875635],[0.02002699381912176]
59497,[3.546446922861235e-07],[-0.0020579970869949076],[-0.0014018295764061108],[-0.007300881657764066],[-0.03146871227937833],[-0.020452033831389337],[0.03329158862513565],[0.015935259539925847],[0.04890628014856153]
59498,[2.5357800228206475e-07],[0.0053113430154788565],[-0.0011149248775742739],[-0.014596045097233676],[0.014122369938013043],[0.0014210745834797758],[-0.06022294724231148],[-0.02169845936934081],[0.002349383607339003]


In [189]:
print(len(data_dict["Noise"]))
new_data = pd.DataFrame(data_dict)
new_data

59500


,Noise
0,[4.794858948905414e-06]
1,[-1.7004199106781814e-05]
2,[1.3722867716429112e-05]
3,[5.186161100491352e-07]
4,[-2.9333515056164692e-05]
...,...
59495,[-0.020959493411205826]
59496,[-0.031819594795125805]
59497,[-0.01686977292020715]
59498,[-0.010283744032886843]


In [190]:
new_dict_X = {f"{n} Non-Noise": [] for n in range(9)}
new_dict_y = {"Noise": []}

for n in range(9):
  for i in new_data_non[f"{n} Non-Noise"]:
    new_dict_X[f"{n} Non-Noise"].append(float(i))

for i in new_data["Noise"]:
  new_dict_y["Noise"].append(float(i))

new_df_X = pd.DataFrame(new_dict_X)
new_df_y = pd.DataFrame(new_dict_y)

new_df_X

<ipython-input-190-5c2e85e34d9a>:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  new_dict_X[f"{n} Non-Noise"].append(float(i))
<ipython-input-190-5c2e85e34d9a>:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  new_dict_y["Noise"].append(float(i))


,0 Non-Noise,1 Non-Noise,2 Non-Noise,3 Non-Noise,4 Non-Noise,5 Non-Noise,6 Non-Noise,7 Non-Noise,8 Non-Noise
0,0.000000e+00,2.007906e-06,-1.281737e-06,2.003603e-06,4.358142e-06,-0.000002,-0.000007,0.000006,-0.000003
1,0.000000e+00,-5.913955e-07,2.344012e-06,6.824708e-06,6.569007e-06,0.000003,-0.000003,0.000003,0.000010
2,0.000000e+00,3.314156e-07,-3.893219e-07,4.256934e-06,3.747554e-07,0.000005,0.000008,-0.000004,-0.000015
3,0.000000e+00,8.637198e-07,4.169061e-06,-2.203014e-06,1.328094e-06,-0.000001,0.000002,0.000011,-0.000008
4,0.000000e+00,-9.223603e-08,7.624650e-08,6.365160e-07,2.424486e-06,0.000002,0.000009,-0.000002,-0.000005
...,...,...,...,...,...,...,...,...,...
59495,6.661903e-07,-2.668240e-03,8.169984e-03,1.674077e-02,-2.294162e-03,-0.000270,0.011872,-0.020557,0.002076
59496,4.879257e-07,-4.265291e-03,-1.647221e-02,-1.190998e-02,-7.017095e-04,0.006876,-0.028456,-0.007546,0.020027
59497,3.546447e-07,-2.057997e-03,-1.401830e-03,-7.300882e-03,-3.146871e-02,-0.020452,0.033292,0.015935,0.048906
59498,2.535780e-07,5.311343e-03,-1.114925e-03,-1.459605e-02,1.412237e-02,0.001421,-0.060223,-0.021698,0.002349


In [191]:
new_df_y

,Noise
0,4.794859e-06
1,-1.700420e-05
2,1.372287e-05
3,5.186161e-07
4,-2.933352e-05
...,...
59495,-2.095949e-02
59496,-3.181959e-02
59497,-1.686977e-02
59498,-1.028374e-02


In [192]:
print(f"Non-noise data shape: {new_data_non.shape}, Noise data shape: {new_data.shape}")

Non-noise data shape: (59500, 9), Noise data shape: (59500, 1)


## Розподіл вибірок

In [193]:
X = new_df_X
y = new_df_y

print(f"X shape: {X.shape}, y shape: {y.shape}")

X shape: (59500, 9), y shape: (59500, 1)


In [194]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print(f"X_train size: {X_train.shape}, X_test size: {X_test.shape}, y_train size: {y_train.shape}, y_test size:{y_test.shape}")

X_train size: (47600, 9), X_test size: (11900, 9), y_train size: (47600, 1), y_test size:(11900, 1)


### Побудова NN

In [195]:
from keras import models, layers


model = models.Sequential([
        layers.Dense(1600, activation="relu", input_shape=(9,)),
        layers.Dense(400, activation="relu"),
        layers.Dense(1, activation="sigmoid")
])

In [186]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 1600)              16000     
                                                                 
 dense_41 (Dense)            (None, 400)               640400    
                                                                 
 dense_42 (Dense)            (None, 1)                 401       
                                                                 
Total params: 656801 (2.51 MB)
Trainable params: 656801 (2.51 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [187]:
from keras import optimizers, losses, metrics


model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["mean_squared_error"])


model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=64)

# history = model.fit(
#     X_train,
#     y_train,
#     batch_size=64,
#     epochs=20,
#     validation_split=0.2,
#     verbose=1
# )

Epoch 1/20
744/744 [==============================] - 13s 17ms/step - loss: 0.2073 - mean_squared_error: 0.0040 - val_loss: 0.2030 - val_mean_squared_error: 0.0022
Epoch 2/20
744/744 [==============================] - 12s 16ms/step - loss: 0.1992 - mean_squared_error: 0.0015 - val_loss: 0.2009 - val_mean_squared_error: 0.0016
Epoch 3/20
744/744 [==============================] - 12s 16ms/step - loss: 0.1992 - mean_squared_error: 0.0015 - val_loss: 0.1999 - val_mean_squared_error: 0.0014
Epoch 4/20
744/744 [==============================] - 13s 17ms/step - loss: 0.1991 - mean_squared_error: 0.0015 - val_loss: 0.1999 - val_mean_squared_error: 0.0015
Epoch 5/20
744/744 [==============================] - 12s 16ms/step - loss: 0.1990 - mean_squared_error: 0.0015 - val_loss: 0.2003 - val_mean_squared_error: 0.0014
Epoch 6/20
744/744 [==============================] - 11s 15ms/step - loss: 0.1988 - mean_squared_error: 0.0015 - val_loss: 0.2004 - val_mean_squared_error: 0.0018
Epoch 7/20
744/7